In [1]:
import tensorflow as tf
from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt
from keras_tuner import RandomSearch
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.losses import categorical_crossentropy


     |████████████████████████████████| 98 kB 3.2 MB/s 


In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.cifar10.load_data()

# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0


label_train = keras.utils.to_categorical(label_train, 10)
label_test = keras.utils.to_categorical(label_test, 10)

170508288/170498071 [==============================] - 3s 0us/step


In [4]:
def build_model(hp):
    model = keras.Sequential()
    model.add(Conv2D(hp.Int(f"features_c_{-1}", min_value=32, max_value=256, step=32),
                     3, activation ="relu", input_shape=(32, 32, 3)))

    for i in range(hp.Int("num_conv_layers", 0, 4)):
        model.add(
            Conv2D(
                hp.Int(f"features_c_{i}", min_value=32, max_value=256, step=32),
                3,                
                activation='relu',
            )
        )
        #   model.add(Conv2D(8, kernel_size, activation ="relu", input_shape=(150, 150, 3)))
    model.add(Flatten())

    for i in range(hp.Int("num_dense_layers", 1, 5)):
        model.add(
            Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation='relu',
            )
        )
    model.add(Dense(10, activation="softmax"))
    
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-3, sampling="log")
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss= tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy'])
    return model


    # # Tune the number of units in the first Dense layer
    # # Choose an optimal value between 32-512
    # hp_units = hp.Int('units', min_value=16, max_value=512, step=32)
    # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    # model.add(keras.layers.Dense(10))

    # # Tune the learning rate for the optimizer
    # # Choose an optimal value from 0.01, 0.001, or 0.0001
    # hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

   


In [5]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="cifar",
)


In [6]:
tuner.search_space_summary()
tuner.search(img_train,label_train,epochs=10,validation_data=(img_test,label_test))


Trial 3 Complete [00h 34m 19s]
val_accuracy: 0.7275999784469604

Best val_accuracy So Far: 0.7275999784469604
Total elapsed time: 01h 09m 11s
INFO:tensorflow:Oracle triggered exit


In [7]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
best_model.build(input_shape=(None, 32, 32))
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 160)       4480      
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 128)       184448    
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 192)       221376    
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 224)       387296    
                                                                 
 flatten (Flatten)           (None, 129024)            0         
                                                                 
 dense (Dense)               (None, 256)               33030400  
                                                                 
 dense_1 (Dense)             (None, 320)               8

In [8]:
tuner.results_summary()


Results summary
Results in my_dir/cifar
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
features_c_-1: 160
num_conv_layers: 3
num_dense_layers: 3
units_0: 256
lr: 0.0002580801474874774
features_c_0: 128
features_c_1: 192
features_c_2: 224
features_c_3: 128
units_1: 320
units_2: 480
units_3: 256
units_4: 384
Score: 0.7275999784469604
Trial summary
Hyperparameters:
features_c_-1: 192
num_conv_layers: 4
num_dense_layers: 5
units_0: 160
lr: 0.0001486668133601622
features_c_0: 128
features_c_1: 192
features_c_2: 96
features_c_3: 64
units_1: 320
units_2: 32
units_3: 32
units_4: 32
Score: 0.7060999870300293
Trial summary
Hyperparameters:
features_c_-1: 128
num_conv_layers: 4
num_dense_layers: 2
units_0: 192
lr: 0.0006488638452453321
features_c_0: 32
features_c_1: 32
features_c_2: 32
features_c_3: 32
units_1: 32
Score: 0.6674999892711639


In [9]:
import numpy as np

best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
model = build_model(best_hps[0])
# Fit with the entire dataset.
x_all = np.concatenate((img_train, img_test))
y_all = np.concatenate((label_train, label_test))
model.fit(x=x_all, y=y_all, epochs=1)

1875/1875 [==============================] - 113s 60ms/step - loss: 1.3166 - accuracy: 0.5250
